# Downloading data from FEWS NET USGS

In [2]:
# imports
%load_ext jupyter_black
from pathlib import Path
import os
import requests, zipfile, io
from bs4 import BeautifulSoup
import geopandas as gpd
import rioxarray as rio

In [3]:
data_dir = Path(os.getenv("HAM_DIR"))
shp_dir = Path(os.getenv("AA_DATA_DIR")) / "public/raw/ner/cod_ab/ner_cod_ab/"

In [4]:
ner_shp_adm0 = gpd.read_file(shp_dir / "adm0.shp.zip")
ner_shp_adm0.total_bounds

array([ 0.16625  , 11.69697  , 15.99564  , 23.5331862])

In [5]:
# function for listing files under URL
def list_files(URL, ext):
    page = requests.get(URL, verify=False).text
    soup = BeautifulSoup(page, "html.parser")
    for node in soup.find_all("a"):
        if node.get("href").endswith(ext):
            file_url = [
                URL + "/" + node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
            file_list = [
                node.get("href")
                for node in soup.find_all("a")
                if node.get("href").endswith(ext)
            ]
    return file_list

## NDVI

Temporally Smoothed NDVI - Smoothed 10-day NDVI composite.

eVIIRS NDVI data are stretched (mapped) linearly (to byte values) as follows:

[-1.0, 1.0] -> [0, 200] - Invalid Values: 201 - 255

NDVI = (value - 100) / 100; example: [(150 - 100) / 100 = 0.5 NDVI] 

More information [here](https://earlywarning.usgs.gov/fews/product/903)

In [6]:
ndvi_dir = data_dir / "vegetation/ndvi/"

In [7]:
URL_ndvi = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/fews/web/africa/west/pentadal/eviirs/ndvi/temporallysmoothedndvi/downloads/pentadal/"

ndvi_file_list = list_files(URL=URL_ndvi, ext="zip")

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
for file in ndvi_file_list:
    if file in ndvi_file_list:
        req = requests.get(URL_ndvi + file, verify=False, stream=True)
        with zipfile.ZipFile(io.BytesIO(req.content)) as zObj:
            fileNames = zObj.namelist()
            for fileName in fileNames:
                if fileName.endswith("tif"):
                    content = zObj.open(fileName).read()
                    open(ndvi_dir / fileName, "wb").write(content)
                    input_raster = rio.open_rasterio(ndvi_dir / fileName)
                    input_value = input_raster.load()
                    input_raster.close()
                    rast_clip = input_value.rio.clip_box(*ner_shp_adm0.total_bounds)
                    rast_clip.rio.to_raster(ndvi_dir / fileName)

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\hamenv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edcintl.cr.usgs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

## WRSI

In [ ]:
wrsicroplands_dir = data_dir / "water_requirements/wrsi/croplands/"
wrsirangelands_dir = data_dir / "water_requirements/wrsi/rangelands/"

URL_wrsi = ""
wrsi_file_list = list_files(URL=, ext="zip")